In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    LeakyReLU,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理：归一化
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)  # 调整形状以匹配模型输入
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train.reshape(-1, 28, 28, 1))  # 数据增强需要原始形状

# 构建多层感知机（MLP）神经网络模型
model = Sequential(
    [
        Input(shape=(28, 28, 1)),  # 输入层
        
        tf.keras.layers.Flatten(),  # 展平图像
        Dense(2048),  # 隐藏层 1
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(1024),  # 隐藏层 2
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512),  # 隐藏层 3
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256),  # 隐藏层 4
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128),  # 隐藏层 5
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(64),  # 隐藏层 6
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(32),  # 隐藏层 7
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
history = model.fit(
    datagen.flow(x_train.reshape(-1, 28, 28, 1), y_train, batch_size=128),  # 调整批量大小并恢复原始形状
    epochs=200,  # 增加训练轮数
    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test),  # 恢复原始形状
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)  # 使用调整后的测试数据形状
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_mlp_v2.keras")

2025-01-11 20:21:42.787999: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-01-11 20:21:42.788035: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-11 20:21:42.788047: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-11 20:21:42.788076: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-11 20:21:42.788091: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2025-01-11 20:21:44.099342: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


468/469 [============================>.] - ETA: 0s - loss: 2.4503 - accuracy: 0.1533

2025-01-11 20:22:03.997202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 24s 41ms/step - loss: 2.4498 - accuracy: 0.1535 - val_loss: 1.9828 - val_accuracy: 0.2270 - lr: 0.0010
Epoch 2/200
469/469 [==============================] - 18s 38ms/step - loss: 2.0875 - accuracy: 0.2257 - val_loss: 1.7165 - val_accuracy: 0.3000 - lr: 0.0010
Epoch 3/200
469/469 [==============================] - 18s 38ms/step - loss: 1.9327 - accuracy: 0.2859 - val_loss: 1.4588 - val_accuracy: 0.4570 - lr: 0.0010
Epoch 4/200
469/469 [==============================] - 18s 38ms/step - loss: 1.7616 - accuracy: 0.3671 - val_loss: 1.0941 - val_accuracy: 0.5591 - lr: 0.0010
Epoch 5/200
469/469 [==============================] - 18s 38ms/step - loss: 1.5887 - accuracy: 0.4405 - val_loss: 0.8760 - val_accuracy: 0.6948 - lr: 0.0010
Epoch 6/200
469/469 [==============================] - 18s 38ms/step - loss: 1.4744 - accuracy: 0.4890 - val_loss: 0.7988 - val_accuracy: 0.7125 - lr: 0.0010
Epoch 7/200
469/469 [==============================] - 18s 38ms/